In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.ensemble import (RandomForestRegressor, GradientBoostingRegressor,
                              AdaBoostRegressor, HistGradientBoostingRegressor)
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
import joblib
import warnings
warnings.filterwarnings('ignore')

PLEASE UPLOAD ALL FILES ONE BY ONE


In [56]:
from google.colab import files

uploaded = files.upload()

patient = pd.read_csv('patient.csv')

Saving patient.csv to patient (3).csv


In [ ]:
from google.colab import files

uploaded = files.upload()


diagnosis = pd.read_csv('diagnosis.csv')


In [ ]:
from google.colab import files

uploaded = files.upload()


care = pd.read_csv('care.csv')


In [ ]:
from google.colab import files

uploaded = files.upload()

risk = pd.read_csv('risk.csv')


In [ ]:
from google.colab import files

uploaded = files.upload()

visit = pd.read_csv('visit.csv')



In [3]:
df = patient.copy()

In [4]:
df['patient_id'].duplicated().sum()

np.int64(0)

CARE TABLE


In [5]:
# Ensure date columns are datetime objects
date_columns = ['visit_start_dt', 'visit_end_dt', 'follow_up_dt']
for col in date_columns:
    if col in visit.columns:
        visit[col] = pd.to_datetime(visit[col], errors='coerce')

# Find the latest date across all relevant columns
latest_date_in_visit = visit[date_columns].max().max()

print(f"The latest date in the visit data is: {latest_date_in_visit}")
# Convert 'hot_spotter_identified_at' to datetime objects
df['hot_spotter_identified_at'] = pd.to_datetime(df['hot_spotter_identified_at'], errors='coerce')

# Calculate the time difference in days since the hotspot was identified
# Use the latest_date_in_visit found previously as the reference
df['time_since_hotspot_identified'] = (latest_date_in_visit - df['hot_spotter_identified_at']).dt.days

df.drop('hot_spotter_identified_at', axis=1 ,inplace=True )

The latest date in the visit data is: 2025-03-03 00:00:00


In [6]:
# Combine msrmnt_type and msrmnt_sub_type in the care table
care['msrmnt_type_subtype'] = care['msrmnt_type'] + '_' + care['msrmnt_sub_type']

# Group by patient_id and aggregate the combined string, handling potential NaNs
care_aggregated = care.groupby('patient_id')['msrmnt_type_subtype'].apply(lambda x: '_'.join(x.dropna().unique())).reset_index()

# Merge the aggregated care data with the df DataFrame
df = pd.merge(df, care_aggregated, on='patient_id', how='left')

# Display the updated df DataFrame
display(df.head())

,patient_id,age,hot_spotter_readmission_flag,hot_spotter_chronic_flag,time_since_hotspot_identified,msrmnt_type_subtype
0,276,18,f,f,NaN,NaN
1,309,19,f,f,NaN,NaN
2,327,28,f,f,NaN,NaN
3,333,23,f,f,NaN,NaN
4,344,54,f,f,NaN,NaN


In [7]:
# Group care by patient_id and aggregate care_gap_ind
care_gap_aggregated = care.groupby('patient_id')['care_gap_ind'].apply(lambda x: '_'.join(x.dropna().unique())).reset_index()

# Merge with df
df = pd.merge(df, care_gap_aggregated, on='patient_id', how='left')

# Display the updated df DataFrame
display(df.head())

,patient_id,age,hot_spotter_readmission_flag,hot_spotter_chronic_flag,time_since_hotspot_identified,msrmnt_type_subtype,care_gap_ind
0,276,18,f,f,NaN,NaN,NaN
1,309,19,f,f,NaN,NaN,NaN
2,327,28,f,f,NaN,NaN,NaN
3,333,23,f,f,NaN,NaN,NaN
4,344,54,f,f,NaN,NaN,NaN


Diagnosis Table

In [8]:
# Get unique condition names from the diagnosis table
unique_conditions = diagnosis['condition_name'].unique()

# Create a new DataFrame with patient_id and a column of ones
diagnosis_binary = diagnosis[['patient_id', 'condition_name']].copy()
diagnosis_binary['has_condition'] = 1

# Pivot the table to get unique conditions as columns
diagnosis_pivot = diagnosis_binary.pivot_table(
    index='patient_id',
    columns='condition_name',
    values='has_condition',
    fill_value=0
).reset_index()

# Merge the new binary columns with the df DataFrame
df = pd.merge(df, diagnosis_pivot, on='patient_id', how='left')

# Fill NaN values (for patients not in diagnosis) with 0
for condition in unique_conditions:
    df[condition] = df[condition].fillna(0)

# ---- NEW CODE: Calculate chronic_ratio ----
df['chronic_ratio'] = df[unique_conditions].sum(axis=1) / len(unique_conditions)

# Display updated DataFrame
display(df.head())


,patient_id,age,hot_spotter_readmission_flag,hot_spotter_chronic_flag,time_since_hotspot_identified,msrmnt_type_subtype,care_gap_ind,CANCER,DIABETES,HYPERTENSION,chronic_ratio
0,276,18,f,f,NaN,NaN,NaN,0.0,0.0,0.0,0.000000
1,309,19,f,f,NaN,NaN,NaN,0.0,0.0,0.0,0.000000
2,327,28,f,f,NaN,NaN,NaN,0.0,0.0,0.0,0.000000
3,333,23,f,f,NaN,NaN,NaN,0.0,0.0,0.0,0.000000
4,344,54,f,f,NaN,NaN,NaN,0.0,0.0,1.0,0.333333


VISIT TABLE

In [9]:
# --- Unique diagnosis values (drop NaN early)
unique_visit_diag = visit['prncpl_diag_nm'].dropna().unique()

# --- Build patient × diagnosis multi-hot (presence = 1)
visit_binary = (
    visit[['patient_id', 'prncpl_diag_nm']]
      .dropna(subset=['prncpl_diag_nm'])
      .assign(has_diag=1)
)

visit_pivot = (
    visit_binary
      .pivot_table(
          index='patient_id',
          columns='prncpl_diag_nm',
          values='has_diag',
          aggfunc='max',        # presence if ever seen
          fill_value=0
      )
      .rename_axis(None, axis=1)
)

# Optional: add a prefix to avoid collisions
visit_pivot = visit_pivot.add_prefix('dx_')

# Make it small: store as uint8 (or bool)
visit_pivot = visit_pivot.astype('uint8')

# --- Merge into your main df on patient_id
df = df.merge(visit_pivot.reset_index(), on='patient_id', how='left')

# Columns that were absent for some patients will be NaN after merge; fill them:
dx_cols = [c for c in df.columns if c.startswith('dx_')]
df[dx_cols] = df[dx_cols].fillna(0).astype('uint8')

# --- Ratios / totals based on distinct diagnoses present
df['visit_diag_ratio'] = df[dx_cols].sum(axis=1) / len(dx_cols)
df['total_visit_diagnoses'] = df[dx_cols].sum(axis=1)

# peek
display(df.head())


,patient_id,age,hot_spotter_readmission_flag,hot_spotter_chronic_flag,time_since_hotspot_identified,msrmnt_type_subtype,care_gap_ind,CANCER,DIABETES,HYPERTENSION,...,dx_Ventricular premature depolarization,"dx_Ventricular tachycardia, unspecified","dx_Viral infection, unspecified","dx_Viral intestinal infection, unspecified",dx_Weakness,dx_Wheezing,dx_Zoster meningitis,dx_Zoster without complications,visit_diag_ratio,total_visit_diagnoses
0,276,18,f,f,NaN,NaN,NaN,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.000000,0
1,309,19,f,f,NaN,NaN,NaN,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.001661,1
2,327,28,f,f,NaN,NaN,NaN,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.000000,0
3,333,23,f,f,NaN,NaN,NaN,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.003322,2
4,344,54,f,f,NaN,NaN,NaN,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0.001661,1


In [10]:
import pandas as pd
import re

# -----------------------------
# 1) BINNING RULES
# -----------------------------
rules = [
    ('Respiratory Infection', r'infection|pneumonia|bronchitis|pharyngitis|laryngitis|tracheitis|tonsillitis|sinusitis|nasopharyngitis|flu|influenza|covid'),
    ('Lower Respiratory', r'pneumonia|bronchitis|bronchiolitis|wheezing|asthma|copd'),
    ('Upper Respiratory', r'cough|cold|throat|upper respiratory|pharyngitis|laryngitis'),
    ('ENT', r'otitis|ear|sinus|sinusitis|epistaxis|rhinitis|cerumen|tonsil|throat|pharynx|larynx'),
    ('Musculoskeletal', r'strain|sprain|fracture|contusion|myalgia|arthritis|back pain|shoulder|wrist|knee|hip|ligament|muscle|joint|tendon|osteo|sciatica'),
    ('Injury/Wound', r'injury|wound|laceration|open wound|foreign body|bite|burn|abrasion|crush|trauma|contusion|dislocation|fracture|amputation'),
    ('Skin/Soft Tissue', r'cellulitis|abscess|furuncle|erythematous|dermatitis|rash|swelling|urticaria|cyst|ulcer|bite|laceration|wound|pruritus'),
    ('GU', r'cystitis|urinary|bladder|hematuria|pyelonephritis|prostatitis|incontinence|urethritis'),
    ('GI', r'abdominal|gastro|nausea|vomiting|epigastric|colitis|diarrhea|constipation|appendicitis|gastritis|pancreatitis|hepatitis|hernia|peritonitis|cholecystitis|gallbladder|bleeding|hemorrhoids|gerd|reflux|diverticulitis'),
    ('Neurology/Psych', r'headache|migraine|dizz|giddiness|vertigo|syncope|collapse|seizure|epilepsy|paralysis|stroke|tremor|neuro|disorder|depression|anxiety|mood|insomnia|sleep'),
    ('Cardiovascular', r'chest pain|palpitations|hypertension|tachy|arrhythmia|heart failure|infarction|angina|embolism|thrombosis|atherosclerosis|hypotension|stemi|nstemi'),
    ('Obstetric/Gyne', r'pregnancy|labor|childbirth|preterm|vaginitis|menstruation|miscarriage|abortion|perineal|postpartum|uterovaginal|ovarian|endometriosis|fetal|maternal care|gestational'),
    ('Endocrine/Metabolic', r'diabetes|thyroid|metabolic|nutritional|obesity|hypoglycemia|ketoacidosis|electrolyte|hypokalemia|hyperglycemia'),
    ('Eye', r'conjunctivitis|hordeolum|chalazion|stye|blepharitis|cataract|glaucoma|corneal|iridocyclitis|retinal'),
    ('Other Infection', r'viral|bacterial|abscess|sepsis|tuberculosis|mononucleosis|infection'),
    ('Allergy/Immune', r'allergy|urticaria|anaphylaxis|angioedema|immune|contact dermatitis'),
    ('Pain', r'pain'),
    ('Other', r'fever|malaise|fatigue|unspecified|other|abnormal|screening|observation|follow-up'),
]

def bin_diagnosis(text):
    if pd.isnull(text):
        return 'Other'
    t = text.lower()
    for cat, pat in rules:
        if re.search(pat, t):
            return cat
    return 'Other'

# -----------------------------
# 2) APPLY BINNING
# -----------------------------
visit['diag_bin'] = visit['prncpl_diag_nm'].apply(bin_diagnosis)

# -----------------------------
# 3) One-hot diag bins + visit types
# -----------------------------
diag_dummies = pd.get_dummies(visit['diag_bin'], prefix='diag', dtype=int)
visit_type_dummies = pd.get_dummies(visit['visit_type'], prefix='visit_type', dtype=int)

# -----------------------------
# 4) readmission mapping
# -----------------------------
def encode_flag(value):
    if pd.isna(value):
        return 0  # or -1 if you prefer
    return int(bool(value))

visit['readmsn_ind'] = visit['readmsn_ind'].apply(encode_flag)

# -----------------------------
# 5) Combine + aggregate
# -----------------------------
df_for_agg = pd.concat([visit[['patient_id', 'readmsn_ind']], visit_type_dummies, diag_dummies], axis=1)

# dynamic aggregations
aggregations = {
    **{c: (c, 'sum') for c in visit_type_dummies.columns},
    **{c: (c, 'sum') for c in diag_dummies.columns},
    'visit_count': ('readmsn_ind', 'count'),
    'total_readmissions': ('readmsn_ind', 'sum')
}

patient_summary = df_for_agg.groupby('patient_id').agg(**aggregations).reset_index()

# convert visit type to indicator
for c in visit_type_dummies.columns:
    patient_summary[c] = (patient_summary[c] > 0).astype(int)


In [11]:
df = df.merge(patient_summary, on='patient_id', how='left')

# -----------------------------
# ✅ FIX: fill NaNs with 0
# -----------------------------
fill_cols = [c for c in patient_summary.columns if c != 'patient_id']
df[fill_cols] = df[fill_cols].fillna(0).astype(int)


In [12]:
df.head()

,patient_id,age,hot_spotter_readmission_flag,hot_spotter_chronic_flag,time_since_hotspot_identified,msrmnt_type_subtype,care_gap_ind,CANCER,DIABETES,HYPERTENSION,...,diag_Neurology/Psych,diag_Obstetric/Gyne,diag_Other,diag_Other Infection,diag_Pain,diag_Respiratory Infection,diag_Skin/Soft Tissue,diag_Upper Respiratory,visit_count,total_readmissions
0,276,18,f,f,NaN,NaN,NaN,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,309,19,f,f,NaN,NaN,NaN,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,1,1
2,327,28,f,f,NaN,NaN,NaN,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,333,23,f,f,NaN,NaN,NaN,0.0,0.0,0.0,...,1,0,1,0,0,0,0,0,2,2
4,344,54,f,f,NaN,NaN,NaN,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,1,1


In [13]:
# --- ensure date columns are datetime ---
visit = visit.copy()
for col in ['visit_end_dt', 'follow_up_dt']:
    if col in visit.columns:
        visit[col] = pd.to_datetime(visit[col], errors='coerce')

# --- 1) one-hot / binary columns for visit_type (similar to diagnosis pivot) ---
unique_visit_types = visit['visit_type'].dropna().unique()

visit_type_binary = visit[['patient_id', 'visit_type']].copy()
visit_type_binary['has_visit_type'] = 1

visit_type_pivot = visit_type_binary.pivot_table(
    index='patient_id',
    columns='visit_type',
    values='has_visit_type',
    fill_value=0
).reset_index()

# If any visit_type column names collide with df, you can keep as-is or rename:
visit_type_cols = [c for c in visit_type_pivot.columns if c != 'patient_id']
# optional: add prefix
visit_type_pivot = visit_type_pivot.rename(columns={c: f"visit_type__{c}" for c in visit_type_cols})
visit_type_cols = [f"visit_type__{c}" for c in visit_type_cols]

# --- 2) total visits per patient ---
visits_count = visit.groupby('patient_id').size().reset_index(name='total_visits')

# --- 3) min / max difference in days between follow_up_dt and visit_end_dt per patient ---
if {'visit_end_dt', 'follow_up_dt'}.issubset(visit.columns):
    visit['diff_days'] = (visit['follow_up_dt'] - visit['visit_end_dt']).dt.days
else:
    visit['diff_days'] = pd.NA

diff_stats = visit.groupby('patient_id').agg(
    min_diff_days=('diff_days', 'min'),
    max_diff_days=('diff_days', 'max'),
    mean_diff_days=('diff_days', 'mean')  # optional
).reset_index()

# --- Merge everything into df ---
df = df.merge(visit_type_pivot, on='patient_id', how='left')
df = df.merge(visits_count, on='patient_id', how='left')
df = df.merge(diff_stats, on='patient_id', how='left')

# --- Fill NaNs ---
for col in visit_type_cols:
    df[col] = df[col].fillna(0)

df['total_visits'] = df['total_visits'].fillna(0).astype(int)

# --- 4) visit_type_ratio: proportion of unique visit types patient has ---
num_visit_types = len(unique_visit_types) if len(unique_visit_types) > 0 else 1
df['visit_type_ratio'] = df[visit_type_cols].sum(axis=1) / num_visit_types

# --- final: show head ---
display(df.head())


,patient_id,age,hot_spotter_readmission_flag,hot_spotter_chronic_flag,time_since_hotspot_identified,msrmnt_type_subtype,care_gap_ind,CANCER,DIABETES,HYPERTENSION,...,visit_count,total_readmissions,visit_type__ER,visit_type__INPATIENT,visit_type__URGENT CARE,total_visits,min_diff_days,max_diff_days,mean_diff_days,visit_type_ratio
0,276,18,f,f,NaN,NaN,NaN,0.0,0.0,0.0,...,0,0,0.0,0.0,0.0,0,NaN,NaN,NaN,0.000000
1,309,19,f,f,NaN,NaN,NaN,0.0,0.0,0.0,...,1,1,0.0,0.0,1.0,1,NaN,NaN,NaN,0.333333
2,327,28,f,f,NaN,NaN,NaN,0.0,0.0,0.0,...,0,0,0.0,0.0,0.0,0,NaN,NaN,NaN,0.000000
3,333,23,f,f,NaN,NaN,NaN,0.0,0.0,0.0,...,2,2,1.0,1.0,0.0,2,NaN,NaN,NaN,0.666667
4,344,54,f,f,NaN,NaN,NaN,0.0,0.0,1.0,...,1,1,1.0,0.0,0.0,1,NaN,NaN,NaN,0.333333


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2001 entries, 0 to 2000
Columns: 646 entries, patient_id to visit_type_ratio
dtypes: float64(13), int64(26), object(4), uint64(1), uint8(602)
memory usage: 1.8+ MB


In [15]:
# --- Count readmission 't' and 'f' per patient ---
readmission_stats = (
    visit.groupby(['patient_id', 'readmsn_ind'])
    .size()
    .unstack(fill_value=0)  # creates columns 'f' and 't'
    .reset_index()
)

# Ensure both columns exist (some patients may have only 't' or only 'f')
for col in ['t', 'f']:
    if col not in readmission_stats.columns:
        readmission_stats[col] = 0

# --- Merge only counts into df ---
df = df.merge(
    readmission_stats[['patient_id', 't', 'f']],
    on='patient_id',
    how='left'
)

# Rename columns for clarity
df = df.rename(columns={
    't': 'readmission_true_count',
    'f': 'readmission_false_count'
})

# Fill missing values with 0
df[['readmission_true_count', 'readmission_false_count']] = (
    df[['readmission_true_count', 'readmission_false_count']].fillna(0)
)

# --- Display updated DataFrame ---
display(df.head())


,patient_id,age,hot_spotter_readmission_flag,hot_spotter_chronic_flag,time_since_hotspot_identified,msrmnt_type_subtype,care_gap_ind,CANCER,DIABETES,HYPERTENSION,...,visit_type__ER,visit_type__INPATIENT,visit_type__URGENT CARE,total_visits,min_diff_days,max_diff_days,mean_diff_days,visit_type_ratio,readmission_true_count,readmission_false_count
0,276,18,f,f,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0,NaN,NaN,NaN,0.000000,0.0,0.0
1,309,19,f,f,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,1.0,1,NaN,NaN,NaN,0.333333,0.0,0.0
2,327,28,f,f,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0,NaN,NaN,NaN,0.000000,0.0,0.0
3,333,23,f,f,NaN,NaN,NaN,0.0,0.0,0.0,...,1.0,1.0,0.0,2,NaN,NaN,NaN,0.666667,0.0,0.0
4,344,54,f,f,NaN,NaN,NaN,0.0,0.0,1.0,...,1.0,0.0,0.0,1,NaN,NaN,NaN,0.333333,0.0,0.0


In [16]:
# --- Count total follow_up_date entries per patient ---
follow_up_counts = (
    visit.groupby('patient_id')['follow_up_dt']
    .count()  # counts non-null follow_up_date values
    .reset_index(name='total_followups')
)

# --- Merge this count into df --
df = df.merge(
    follow_up_counts,
    on='patient_id',
    how='left'
)

# --- Fill missing values with 0 (for patients with no follow-ups) ---
df['total_followups'] = df['total_followups'].fillna(0).astype(int)

# --- Display updated DataFrame ---
display(df.head())


,patient_id,age,hot_spotter_readmission_flag,hot_spotter_chronic_flag,time_since_hotspot_identified,msrmnt_type_subtype,care_gap_ind,CANCER,DIABETES,HYPERTENSION,...,visit_type__INPATIENT,visit_type__URGENT CARE,total_visits,min_diff_days,max_diff_days,mean_diff_days,visit_type_ratio,readmission_true_count,readmission_false_count,total_followups
0,276,18,f,f,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0,NaN,NaN,NaN,0.000000,0.0,0.0,0
1,309,19,f,f,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,1.0,1,NaN,NaN,NaN,0.333333,0.0,0.0,0
2,327,28,f,f,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0,NaN,NaN,NaN,0.000000,0.0,0.0,0
3,333,23,f,f,NaN,NaN,NaN,0.0,0.0,0.0,...,1.0,0.0,2,NaN,NaN,NaN,0.666667,0.0,0.0,0
4,344,54,f,f,NaN,NaN,NaN,0.0,0.0,1.0,...,0.0,0.0,1,NaN,NaN,NaN,0.333333,0.0,0.0,0


In [17]:
df['follow_up_ratio'] = df['total_followups']/df['total_visits']

In [18]:
df.columns

Index(['patient_id', 'age', 'hot_spotter_readmission_flag',
       'hot_spotter_chronic_flag', 'time_since_hotspot_identified',
       'msrmnt_type_subtype', 'care_gap_ind', 'CANCER', 'DIABETES',
       'HYPERTENSION',
       ...
       'visit_type__URGENT CARE', 'total_visits', 'min_diff_days',
       'max_diff_days', 'mean_diff_days', 'visit_type_ratio',
       'readmission_true_count', 'readmission_false_count', 'total_followups',
       'follow_up_ratio'],
      dtype='object', length=650)

In [19]:
df

,patient_id,age,hot_spotter_readmission_flag,hot_spotter_chronic_flag,time_since_hotspot_identified,msrmnt_type_subtype,care_gap_ind,CANCER,DIABETES,HYPERTENSION,...,visit_type__URGENT CARE,total_visits,min_diff_days,max_diff_days,mean_diff_days,visit_type_ratio,readmission_true_count,readmission_false_count,total_followups,follow_up_ratio
0,276,18,f,f,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,0,NaN,NaN,NaN,0.000000,0.0,0.0,0,NaN
1,309,19,f,f,NaN,NaN,NaN,0.0,0.0,0.0,...,1.0,1,NaN,NaN,NaN,0.333333,0.0,0.0,0,0.0
2,327,28,f,f,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,0,NaN,NaN,NaN,0.000000,0.0,0.0,0,NaN
3,333,23,f,f,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,2,NaN,NaN,NaN,0.666667,0.0,0.0,0,0.0
4,344,54,f,f,NaN,NaN,NaN,0.0,0.0,1.0,...,0.0,1,NaN,NaN,NaN,0.333333,0.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,81925,41,f,f,NaN,NaN,NaN,0.0,0.0,0.0,...,1.0,2,NaN,NaN,NaN,0.666667,0.0,0.0,0,0.0
1997,81942,61,f,f,NaN,NaN,NaN,1.0,0.0,1.0,...,0.0,0,NaN,NaN,NaN,0.000000,0.0,0.0,0,NaN
1998,82069,48,f,f,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,1,NaN,NaN,NaN,0.333333,0.0,0.0,0,0.0
1999,82104,57,f,f,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,0,NaN,NaN,NaN,0.000000,0.0,0.0,0,NaN


In [20]:
# Check for null values in each column and display columns with NaNs
nan_columns = df.columns[df.isnull().any()].tolist()
print("Columns with NaN values:")
print(nan_columns)

Columns with NaN values:
['time_since_hotspot_identified', 'msrmnt_type_subtype', 'care_gap_ind', 'min_diff_days', 'max_diff_days', 'mean_diff_days', 'follow_up_ratio']


In [21]:
# Identify the one-hot encoded visit type columns
visit_type_cols = [col for col in df.columns if col.startswith('visit_type__')]

# Calculate the weighted visits for each visit type
for col in visit_type_cols:
    df[f'{col}_weighted'] = df[col] * df['total_visits']

# Create a single column with the sum of weighted visits for each patient
weighted_visit_cols = [f'{col}_weighted' for col in visit_type_cols]
df['weighted_visits'] = df[weighted_visit_cols].sum(axis=1)

# Display the updated DataFrame with the new weighted columns and the total weighted_visits
display(df[['patient_id', 'total_visits'] + visit_type_cols + weighted_visit_cols + ['weighted_visits']].head())

,patient_id,total_visits,visit_type__ER,visit_type__INPATIENT,visit_type__URGENT CARE,visit_type__ER_weighted,visit_type__INPATIENT_weighted,visit_type__URGENT CARE_weighted,weighted_visits
0,276,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,309,1,0.0,0.0,1.0,0.0,0.0,1.0,1.0
2,327,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,333,2,1.0,1.0,0.0,2.0,2.0,0.0,4.0
4,344,1,1.0,0.0,0.0,1.0,0.0,0.0,1.0


In [22]:
def handle_missing_values(df):
    """
    Intelligently handle missing values based on feature type and medical context
    """
    # Create flags for missing data (often meaningful in medical context)
    df['has_care_data'] = df['msrmnt_type_subtype'].notna().astype(int)
    df['has_care_gap_data'] = df['care_gap_ind'].notna().astype(int)

    # For numerical features, use median instead of 0
    numerical_cols = ['min_diff_days', 'max_diff_days', 'mean_diff_days',
                      'time_since_hotspot_identified', 'weighted_visits']
    for col in numerical_cols:
        if col in df.columns:
            median_val = df[col].median()
            df[col] = df[col].fillna(median_val)
            print(f"  ✓ Filled {col} with median: {median_val:.2f}")

    # For follow_up_ratio, 0 is meaningful (no follow-up)
    if 'follow_up_ratio' in df.columns:
        df['follow_up_ratio'] = df['follow_up_ratio'].fillna(0)
        print(f"  ✓ Filled follow_up_ratio with 0")

    # Categorical - keep as is but add meaningful label
    if 'msrmnt_type_subtype' in df.columns:
        df['msrmnt_type_subtype'] = df['msrmnt_type_subtype'].fillna('no_screening')

    if 'care_gap_ind' in df.columns:
        df['care_gap_ind'] = df['care_gap_ind'].fillna('no_data')

    print("✓ Missing values handled")
    return df

def create_medical_features(df):
    """
    Create clinically meaningful features based on medical knowledge
    """
    print("Creating medical features...")

    # 1. Comorbidity score (weighted by severity)
    if all(col in df.columns for col in ['DIABETES', 'HYPERTENSION', 'CANCER']):
        df['comorbidity_score'] = (
            df['DIABETES'] * 1 +      # Moderate severity
            df['HYPERTENSION'] * 1 +  # Moderate severity
            df['CANCER'] * 3         # High severity
        )
        print("  ✓ Comorbidity score created")

    # 2. Healthcare utilization intensity
    if all(col in df.columns for col in ['visit_type_URGENTCARE', 'visit_type_ER', 'visit_type_INPATIENT']):
        df['utilization_intensity'] = (
            df['visit_type_URGENTCARE']*2 +
            df['visit_type__ER'] * 3 +       # ER visits weighted heavily
            df['visit_type__INPATIENT'] * 5  # Inpatient visits significant
        )
        print("  ✓ Utilization intensity created")

    # 3. Patient engagement score
    if all(col in df.columns for col in ['total_followups', 'follow_up_ratio']):
        df['patient_engagement'] = (
            (df['total_followups'] > 0).astype(int) +
            (df['follow_up_ratio'] > 0.5).astype(int) +
            (df['care_gap_ind'] != 'no_data').astype(int)
        )
        print("  ✓ Patient engagement score created")

    # 4. Age risk categories (standard geriatric categories)
    if 'age' in df.columns:
        df['age_risk_category'] = pd.cut(
            df['age'],
            bins=[0, 18, 45, 65, 100],
            labels=['pediatric', 'adult', 'senior', 'geriatric']
        )
        # Convert to dummy variables
        age_dummies = pd.get_dummies(df['age_risk_category'], prefix='age_cat')
        df = pd.concat([df, age_dummies], axis=1)
        df = df.drop('age_risk_category', axis=1)
        print("  ✓ Age risk categories created")

    # 5. Acute vs Chronic care ratio
    if all(col in df.columns for col in ['total_vist_actue', 'chronic_ratio']):
        df['acute_chronic_ratio'] = df['total_vist_actue'] / (df['chronic_ratio'] + 0.01)
        print("  ✓ Acute/chronic ratio created")

    # 6. Care continuity (inverse of days between visits)
    if 'mean_diff_days' in df.columns:
        df['care_continuity_score'] = np.where(
            df['mean_diff_days'] > 0,
            1 / (df['mean_diff_days'] + 1),
            0
        )
        print("  ✓ Care continuity score created")

    # 7. High-risk flags
    if 'age' in df.columns:
        df['is_high_risk_age'] = (df['age'] >= 65).astype(int)
        print("  ✓ High-risk age flag created")

    if 'comorbidity_score' in df.columns:
        df['is_high_comorbidity'] = (df['comorbidity_score'] >= 2).astype(int)
        print("  ✓ High comorbidity flag created")

    print("✓ All medical features created")
    return df


def stratify_risk(predictions, percentiles=[50, 75, 90]):
    """
    Categorize patients into actionable risk tiers
    """
    thresholds = np.percentile(predictions, percentiles)

    def assign_risk(score):
        if score < thresholds[0]:
            return 'Low Risk'
        elif score < thresholds[1]:
            return 'Moderate Risk'
        elif score < thresholds[2]:
            return 'High Risk'
        else:
            return 'Critical Risk'

    return np.array([assign_risk(s) for s in predictions])

print("✓ Utility functions defined")


✓ Utility functions defined


In [23]:
df = handle_missing_values(df)
df = create_medical_features(df)



  ✓ Filled min_diff_days with median: 6.00
  ✓ Filled max_diff_days with median: 6.00
  ✓ Filled mean_diff_days with median: 6.00
  ✓ Filled time_since_hotspot_identified with median: 30.00
  ✓ Filled weighted_visits with median: 0.00
  ✓ Filled follow_up_ratio with 0
✓ Missing values handled
Creating medical features...
  ✓ Comorbidity score created
  ✓ Patient engagement score created
  ✓ Age risk categories created
  ✓ Care continuity score created
  ✓ High-risk age flag created
  ✓ High comorbidity flag created
✓ All medical features created


In [24]:
# Drop rows with any NaN values
df = df.dropna(axis=1)

# Check for null values in each column after dropping NaNs and display columns with NaNs
nan_columns = df.columns[df.isnull().any()].tolist()
print("Columns with NaN values after dropping rows:")
print(nan_columns)

# Display the updated DataFrame head
display(df.head())

Columns with NaN values after dropping rows:
[]


,patient_id,age,hot_spotter_readmission_flag,hot_spotter_chronic_flag,time_since_hotspot_identified,msrmnt_type_subtype,care_gap_ind,CANCER,DIABETES,HYPERTENSION,...,has_care_gap_data,comorbidity_score,patient_engagement,age_cat_pediatric,age_cat_adult,age_cat_senior,age_cat_geriatric,care_continuity_score,is_high_risk_age,is_high_comorbidity
0,276,18,f,f,30.0,no_screening,no_data,0.0,0.0,0.0,...,0,0.0,0,True,False,False,False,0.142857,0,0
1,309,19,f,f,30.0,no_screening,no_data,0.0,0.0,0.0,...,0,0.0,0,False,True,False,False,0.142857,0,0
2,327,28,f,f,30.0,no_screening,no_data,0.0,0.0,0.0,...,0,0.0,0,False,True,False,False,0.142857,0,0
3,333,23,f,f,30.0,no_screening,no_data,0.0,0.0,0.0,...,0,0.0,0,False,True,False,False,0.142857,0,0
4,344,54,f,f,30.0,no_screening,no_data,0.0,0.0,1.0,...,0,1.0,0,False,False,True,False,0.142857,0,0


In [25]:
#import pandas as pd
#Merge data and risk on patient_id
#data = pd.merge(df, risk, on='patient_id', how='left')

# Display the updated data DataFrame
#display(data.head())

data = df.copy()

In [26]:
cols_to_keep = [
    'age', 'comorbidity_score', 'chronic_ratio', 'HYPERTENSION',
    'age_cat_senior', 'is_high_comorbidity', 'DIABETES', 'patient_engagement',
    'visit_type_ER_weighted', 'is_high_risk_age', 'visit_type_ER', 'visit_type_ER_',
    'msrmnt_type_subtype_SCREENING_COLORECTAL_CANCER', 'weighted_visits',
    'visit_type_ratio', 'visit_diag_ratio', 'total_visits', 'visit_count',
    'total_readmissions', 'age_cat_geriatric', 'total_followups', 'follow_up_ratio',
    'visit_type_INPATIENT_weighted', 'visit_type_INPATIENT', 'CANCER', 'diag_Other',
    'hot_spotter_chronic_flag_t', 'diag_Neurology/Psych',
    'msrmnt_type_subtype_SCREENING_COLORECTAL_CANCER_SCREENING_BREAST_CANCER',
    'diag_Cardiovascular', 'diag_GI',
    'msrmnt_type_subtype_SCREENING_BREAST_CANCER_SCREENING_COLORECTAL_CANCER',
    'diag_Skin/Soft Tissue', 'hot_spotter_chronic_flag_f', 'age_cat_adult',
    'msrmnt_type_subtype_no_screening', 'age_cat_pediatric'
]


In [27]:
len(cols_to_keep)

37

In [28]:
data = data[[c for c in cols_to_keep if c in data.columns]]


In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2001 entries, 0 to 2000
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   age                    2001 non-null   int64  
 1   comorbidity_score      2001 non-null   float64
 2   chronic_ratio          2001 non-null   float64
 3   HYPERTENSION           2001 non-null   float64
 4   age_cat_senior         2001 non-null   bool   
 5   is_high_comorbidity    2001 non-null   int64  
 6   DIABETES               2001 non-null   float64
 7   patient_engagement     2001 non-null   int64  
 8   is_high_risk_age       2001 non-null   int64  
 9   visit_type_ER          2001 non-null   int64  
 10  weighted_visits        2001 non-null   float64
 11  visit_type_ratio       2001 non-null   float64
 12  visit_diag_ratio       2001 non-null   float64
 13  total_visits           2001 non-null   int64  
 14  visit_count            2001 non-null   int64  
 15  tota

In [30]:
missing_cols = [c for c in cols_to_keep if c not in data.columns]
missing_cols


['visit_type_ER_weighted',
 'visit_type_ER_',
 'msrmnt_type_subtype_SCREENING_COLORECTAL_CANCER',
 'visit_type_INPATIENT_weighted',
 'hot_spotter_chronic_flag_t',
 'msrmnt_type_subtype_SCREENING_COLORECTAL_CANCER_SCREENING_BREAST_CANCER',
 'msrmnt_type_subtype_SCREENING_BREAST_CANCER_SCREENING_COLORECTAL_CANCER',
 'hot_spotter_chronic_flag_f',
 'msrmnt_type_subtype_no_screening']

In [31]:
data.head()

,age,comorbidity_score,chronic_ratio,HYPERTENSION,age_cat_senior,is_high_comorbidity,DIABETES,patient_engagement,is_high_risk_age,visit_type_ER,...,follow_up_ratio,visit_type_INPATIENT,CANCER,diag_Other,diag_Neurology/Psych,diag_Cardiovascular,diag_GI,diag_Skin/Soft Tissue,age_cat_adult,age_cat_pediatric
0,18,0.0,0.000000,0.0,False,0,0.0,0,0,0,...,0.0,0,0.0,0,0,0,0,0,False,True
1,19,0.0,0.000000,0.0,False,0,0.0,0,0,0,...,0.0,0,0.0,0,0,0,0,0,True,False
2,28,0.0,0.000000,0.0,False,0,0.0,0,0,0,...,0.0,0,0.0,0,0,0,0,0,True,False
3,23,0.0,0.000000,0.0,False,0,0.0,0,0,1,...,0.0,1,0.0,1,1,0,0,0,True,False
4,54,1.0,0.333333,1.0,True,0,0.0,0,0,1,...,0.0,0,0.0,0,0,0,0,0,False,False


In [38]:
data.shape

(2001, 28)

In [39]:
import pickle

In [42]:
import pickle

with open("model_up.pkl", "rb") as f:
    model = pickle.load(f)


In [43]:
model.predict(data)

array([0.705888  , 0.80949431, 0.89862731, ..., 1.45500719, 1.29251371,
       3.05877672])

In [60]:
import pandas as pd

# Suppose:
# patient_df contains patient_id column
# preds is the numpy array from model.predict(data)

preds = model.predict(data)

predicted_risk_score = pd.DataFrame({
    'patient_id': patient['patient_id'],
    'predicted_risk_score': preds
})

In [57]:
y_actual = risk['risk_score']
y_pred = predicted_risk_score['predicted_value']

NameError: name 'risk' is not defined

In [58]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# ✅ MSE
mse = mean_squared_error(y_actual, y_pred)

# ✅ RMSE
rmse = np.sqrt(mse)

# ✅ MAE
mae = mean_absolute_error(y_actual, y_pred)

# ✅ R²
r2 = r2_score(y_actual, y_pred)

print("R²   :", r2)
print("MSE  :", mse)
print("RMSE :", rmse)
print("MAE  :", mae)


NameError: name 'y_actual' is not defined

In [61]:
# 1. Convert the DataFrame to a CSV file (e.g., named 'risk_scores.csv')
predicted_risk_score.to_csv('Losers_HiLabs_Risk_Score.csv', index=False)

# 2. Trigger the download (This part is crucial for notebooks)
from google.colab import files # If you are in Google Colab
# or: from IPython.display import FileLink # If using standard Jupyter/IPython

# Use 'files.download' for Google Colab
files.download('Losers_HiLabs_Risk_Score.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>